### Basketball and SoccerPenalty event detection

#### PCAMZC321_Group99_EventDetection_BP_PreProcessing_Step-0 - Check file name nomenclature

### This scripts does 
###### 1. 
###### 2. 
###### 3. 
###### 4. 
###### 5. 

### Root folder 
C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO


### Input files location
#### VDO files
##### 1. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\Basketball
##### 2. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\SoccerPenalty

#### Train and Test vdo file names list 
##### 1. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\train_list.csv
##### 2. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\test_list.csv

### Output files location 
##### 1. C:\GG-16-03\CAP4\HumanActivityRecognition\Dataset\VDO\VDOFrames --- all frames 
##### 2. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\Final_frameFile.csv  --- all frames
##### 3. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\trainFrames.csv
##### 4. C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO\testFrames.csv


In [1]:
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
import numpy
print('numpy: {}'.format(np.__version__))
import cv2                                 # for capturing videos
print('cv2: {}'.format(cv2.__version__))

import os, os.path
from pathlib import Path
import math   # for mathematical operations


 




pandas: 1.2.4
numpy: 1.20.1
cv2: 4.5.3


### All VDOs stored in three folders - Basketball  and SoccerPenalty under root foler VDO

In [2]:
Drive = "C:"


# Step-1-Process -  Extracting image frame from VDO 

## Define root folder
root_folder = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting"
print(root_folder)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting


In [3]:
# Set root_folder where all inputs VDO are kept
#root_folder = StepARootFolder
#print(root_folder)



## Example - How to create folders using os library

In [4]:
FramesVdo  = root_folder+"\\VDOFrames"
OutputFiles  = root_folder+"\OutputFiles"


if not os.path.exists(FramesVdo):
   os.makedirs(FramesVdo) 
   
if not os.path.exists(OutputFiles):
   os.makedirs(OutputFiles) 

print(FramesVdo)
print(OutputFiles)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting\VDOFrames
C:/1-GG/CAP4/EventDetection/1-ExampleSetting\OutputFiles


## Example - How to read folders under root folder

In [5]:
sub_folders = [name for name in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, name))]

print(sub_folders)

['Basketball', 'OutputFiles', 'SoccerPenalty', 'VDOFrames']


In [6]:
for child in os.listdir(root_folder):
    if child == "VDOFrames":
        continue
    if child == "OutputFiles":
        continue
        
    print(child)
   
    baseimagefolders = os.path.join(root_folder, child)
    if os.path.isdir(baseimagefolders):

        print(baseimagefolders)

Basketball
C:/1-GG/CAP4/EventDetection/1-ExampleSetting\Basketball
SoccerPenalty
C:/1-GG/CAP4/EventDetection/1-ExampleSetting\SoccerPenalty
test_BSvdo_list.csv


In [7]:
print(baseimagefolders)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting\test_BSvdo_list.csv


In [8]:
file = "C:/1-GG/CAP4/EventDetection/1-ExampleSetting\SoccerPenalty/v_SoccerPenalty_g01_c01.avi"
extension = os.path.splitext(file)[1]
print(extension)

.avi


In [9]:
temp = Path(file).stem # Find file name without extension
print(temp)

v_SoccerPenalty_g01_c01


In [ ]:
#C:/1-GG/CAP4/EventDetection/1-ExampleSetting\SoccerPenalty/v_SoccerPenalty_g01_c01.avi

#  valid extensions of base image
image_path_list = []

valid_vdo_extensions = [".mp4" , ".avi"] #specify vald extensions here
valid_vdo_extensions = [item.lower() for item in valid_vdo_extensions]
     
#create a list all files in directory and
#append files with a vaild extention to image_path_list
for file in os.listdir(baseimagefolders):  ### extract each vdo from a given folder
    #print(file)
    extension = os.path.splitext(file)[1] ### How to extract only extension of a vdo file
    if extension.lower() not in valid_vdo_extensions:
        continue
            
    # image_path_list contains list of all vdo files with full path bame of a given vdo folder
    image_path_list.append(os.path.join(baseimagefolders, file))
 image_path_list

## end of Example

### Start 

#### Main script - OpenCV comes with many powerful video editing functions. 
#### Techniques such as image scanning, face recognition can be accomplished using OpenCV.

##### 1. Take a video as input and break the video into frame by frame and save those frame in a folder "VDOFrames" 
##### 2. "vdo_Framing_Process" function does this splitting indo frames
##### 3. also write a csv ("Final_frameFile.csv") have 3 columns like "FrameFilename" , "FullPathName" and "class"

In [10]:
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
import numpy
print('numpy: {}'.format(np.__version__))
import cv2                                 # for capturing videos
print('cv2: {}'.format(cv2.__version__))

import os, os.path
from pathlib import Path
import math   # for mathematical operations


pandas: 1.2.4
numpy: 1.20.1
cv2: 4.5.3


In [11]:
Drive = "C:"


# Step-1-Process -  Extracting image frame from VDO 

## Define root folder
root_folder = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting"
print(root_folder)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting


## Data pre-processing

##### 1. Reading in the video frame-by-frame.
##### 2. Within a second, a human body does not perform very significant movement. This implies that most of the frames (per second) in our video will be redundant. Therefore, only a subset of all the frames in a video needs to be extracted. This will also reduce the size of the input data which will in turn help the model train faster and can also prevent over-fitting.

### Different strategies can be used for frame extraction as follows

##### Extracting a fixed number of frames from the total frames in the video – say only the first 200 frames (i.e., first 8 seconds of the video).

##### Extracting a fixed number of frames each second from the video – say we need only 5 frames per second from a video whose duration is of 10 seconds. This would return a total of 50 frames from the video. This approach is better in the sense that we are extracting the frames sparsely and uniformly from the entire video.

##### Each frame needs to have the same spatial dimensions (height and width). Hence each frame in a video will have to be resized to the required size.

##### The frames can be converted to grayscale.(not done)

In [12]:
def vdo_Framing_Process(child, imageDir,OutimageDir):
#------------------
    image_height, image_width = 64, 64

    # Following 3 variables - for final csv 
    frame_filename = []
    frame_image    = []
    frame_class    = []
    
    
    suffix = '.jpg'   # Extension of image file name 
    seq = 0
    #  valid extensions of base image
    image_path_list = []
    valid_vdo_extensions = [".mp4" , ".avi"] #specify vald extensions here
    valid_vdo_extensions = [item.lower() for item in valid_vdo_extensions]
     
    #create a list all files in directory and
    #append files with a vaild extention to image_path_list
    for file in os.listdir(imageDir):  ### extract each vdo from a given folder
        #print(file)
        extension = os.path.splitext(file)[1] ### How to extract only extension of a vdo file
        if extension.lower() not in valid_vdo_extensions:
            continue
        # image_path_list contains list of all vdo files with full path bame of a given vdo folder
        image_path_list.append(os.path.join(imageDir, file))
    #print(image_path_list)
    
    #loop through image_path_list to open each vdo - Read each vdo in a loop 
    for vdoFile in image_path_list:
            count = 0
            #print(vdoFile)
            
            cap = cv2.VideoCapture(vdoFile)   # capturing the video from the given path
            ## vdoFile looks like "C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/AllVdo/v_Basketball_g08_c01.avi"
            
            frameRate = cap.get(5) #frame rate
            x=1
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                
                # If Video frame was not successfully read then break the loop
                if (ret != True):
                    break
                    
                # Resize the Frame to fixed Dimensions
                resized_frame = cv2.resize(frame, (image_height, image_width))
    
                #print("we are here")
                if (frameId % math.floor(frameRate) == 0):
                    # storing the frames in a train frame folder 
                    temp = Path(vdoFile).stem # Find file name without extension
                    
                    # temp = os.path.splitext(vdoFile)[0]  ## Find file name without extension
                    #print(temp)
                    filename ="/"+ temp+"_frame%d.jpg" % count;count+=1
                    Outimage = os.path.join(OutimageDir+filename)
                    #print(Outimage)
                    cv2.imwrite(Outimage, resized_frame)   # save frame as JPEG file
                    
                    FIlenameOnly = os.path.basename(vdoFile)
                    
                    ## storing the images and their class in a dataframe
                    frame_filename.append(FIlenameOnly)  # Only filename
                    frame_image.append(Outimage)         # File Name with complete path
                    # creating the class of image        # Class i.e. Basketball , CricketBowling etc
                    frame_class.append(child)
            cap.release()


    # storing the images and their class in a dataframe
    train_data = pd.DataFrame()
    train_data['FrameFilename'] = frame_filename
    train_data['FullPathName'] = frame_image
    train_data['class'] = frame_class
    
    # converting the dataframe into csv file 
    #train_data.to_csv('C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO/train_new.csv',header=True, index=False)
    return train_data

In [13]:
FramesVdo  = root_folder+"\\VDOFrames"
OutputFiles  = root_folder+"\OutputFiles"


if not os.path.exists(FramesVdo):
   os.makedirs(FramesVdo) 
   
if not os.path.exists(OutputFiles):
   os.makedirs(OutputFiles) 

print(FramesVdo)
print(OutputFiles)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting\VDOFrames
C:/1-GG/CAP4/EventDetection/1-ExampleSetting\OutputFiles


# Main script that call vdo_Framing_Process

In [14]:
# Main script that call vdo_Framing_Process 

kount = 0
for child in os.listdir(root_folder):
    if child == "VDOFrames":
        continue
    if child == "OutputFiles":
        continue


    kount = kount + 1
    baseimagefolders = os.path.join(root_folder, child)
    if os.path.isdir(baseimagefolders):
        print("Processing Folder :" , baseimagefolders)
        outImage  = FramesVdo
        
        
        ### Calling vdo_Framing_Process function
        df = vdo_Framing_Process(child , baseimagefolders,outImage)
        
        print(kount)
        if (kount == 1):  ### First iteration - copy function used to create df2 for csv
            df2 = df.copy()
        else:
            df2 = pd.concat([df2 , df])  ### all other iteration concat function to append df2
            #df2.append(df)
        print(df2.shape)
# converting the dataframe into csv file 
#df2.to_csv('C:/GG-16-03/CAP4/HumanActivityRecognition/Dataset/VDO/Final_2vdoframeFile.csv',header=True, index=False)
df2.to_csv(OutputFiles+'/Final_2vdoframeFile.csv',header=True, index=False)


Processing Folder : C:/1-GG/CAP4/EventDetection/1-ExampleSetting\Basketball
1
(705, 3)
Processing Folder : C:/1-GG/CAP4/EventDetection/1-ExampleSetting\SoccerPenalty
2
(1292, 3)


## 21-09 session

### Train and Test (apply) vdo files list given 
#### use following files for train and apply dataset 

######  2, 'C:\1-GG\CAP4\EventDetection\1-ExampleSetting/test_BSvdo_list.csv

##### Need to split train and apply set from Final_frameFile.csv using above files

In [15]:
## Define folder
Drive = "C:"
OutputFiles = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles"
Final_frameFile  = pd.read_csv(OutputFiles+"/Final_2vdoframeFile.csv")




In [16]:
Final_frameFile.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [17]:
print(root_folder)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting


In [18]:
testSet  = pd.read_csv(root_folder+"/test_BSvdo_list.csv")

print(Final_frameFile.shape)

print(testSet.shape)

(1292, 3)
(76, 3)


In [19]:
testSet.head()

,video_name,tag,VDOFileName
0,Basketball/v_Basketball_g01_c01.avi,Basketball,v_Basketball_g01_c01.avi
1,Basketball/v_Basketball_g01_c02.avi,Basketball,v_Basketball_g01_c02.avi
2,Basketball/v_Basketball_g01_c03.avi,Basketball,v_Basketball_g01_c03.avi
3,Basketball/v_Basketball_g01_c04.avi,Basketball,v_Basketball_g01_c04.avi
4,Basketball/v_Basketball_g01_c05.avi,Basketball,v_Basketball_g01_c05.avi


## Extract test set vdo files from Final_frameFile

In [20]:
testSetVDOlist = testSet["VDOFileName"].tolist()

In [21]:
testSetVDOlist

['v_Basketball_g01_c01.avi',
 'v_Basketball_g01_c02.avi',
 'v_Basketball_g01_c03.avi',
 'v_Basketball_g01_c04.avi',
 'v_Basketball_g01_c05.avi',
 'v_Basketball_g01_c06.avi',
 'v_Basketball_g01_c07.avi',
 'v_Basketball_g02_c01.avi',
 'v_Basketball_g02_c02.avi',
 'v_Basketball_g02_c03.avi',
 'v_Basketball_g02_c04.avi',
 'v_Basketball_g02_c05.avi',
 'v_Basketball_g02_c06.avi',
 'v_Basketball_g03_c01.avi',
 'v_Basketball_g03_c02.avi',
 'v_Basketball_g03_c03.avi',
 'v_Basketball_g03_c04.avi',
 'v_Basketball_g03_c05.avi',
 'v_Basketball_g03_c06.avi',
 'v_Basketball_g04_c01.avi',
 'v_Basketball_g04_c02.avi',
 'v_Basketball_g04_c03.avi',
 'v_Basketball_g04_c04.avi',
 'v_Basketball_g05_c01.avi',
 'v_Basketball_g05_c02.avi',
 'v_Basketball_g05_c03.avi',
 'v_Basketball_g05_c04.avi',
 'v_Basketball_g06_c01.avi',
 'v_Basketball_g06_c02.avi',
 'v_Basketball_g06_c03.avi',
 'v_Basketball_g06_c04.avi',
 'v_Basketball_g07_c01.avi',
 'v_Basketball_g07_c02.avi',
 'v_Basketball_g07_c03.avi',
 'v_Basketball

In [22]:
Final_frameFile.columns

Index(['FrameFilename', 'FullPathName', 'class'], dtype='object')

In [23]:
testFrames = Final_frameFile[Final_frameFile['FrameFilename'].isin(testSetVDOlist)]


In [24]:
testFrames.shape

(319, 3)

In [25]:
testFrames.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


## Extract test set vdo files from Final_frameFile

In [26]:
# For train frame - all rows except (~) the test vdo list
trainFrames = Final_frameFile[~Final_frameFile['FrameFilename'].isin(testSetVDOlist)]


In [27]:
trainFrames.shape

(973, 3)

#### Find Common Rows between two Dataframe - trainFrames , testFrames - There should not be common row

In [28]:
## Find Common Rows between two Dataframe - trainFrames , testFrames - There should not be common row

dfcommon = trainFrames.merge(testFrames, how = 'inner' ,indicator=False)
dfcommon

,FrameFilename,FullPathName,class


In [29]:
### Store Train and test set 

In [30]:
trainFrames.to_csv(OutputFiles+'/trainFrames.csv',header=True, index=False)
testFrames.to_csv(OutputFiles+'/testFrames.csv',header=True, index=False)
